<a href="https://colab.research.google.com/github/Amandaagambire/Galamsey-Watch/blob/main/galamsey_watch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Earth Engine**

In [1]:
!pip install earthengine-api geemap folium rasterio
!pip install segmentation-models-pytorch
!pip install -q ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

# **Authenticate & Initialize earth engine**

In [3]:
from google.colab import drive
import ee
import geemap
import folium
import rasterio
from PIL import Image
import numpy as np
import os
import shutil
import pandas as pd
from skimage.color import rgb2gray
from skimage import exposure
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import label, regionprops
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def f(x):
  return x

interact(f, x=10)




# Mount Google Drive
try:
    drive.mount('/content/drive')
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

#Authenticae then initialise google Earth Engine
ee.Authenticate()
ee.Initialize(project = 'ee-agamaamanda')
print("Earth Engine Initialized")


interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Earth Engine Initialized


# **DATA ACQUISITION: Fetching Sentinel -2 Images**

In [ ]:

# set coordinates Region of interest  -> (Ghana)
region = ee.Geometry.Rectangle([-3.3, 4.5, 1.3, 11.2])  # (long, lat)


def preprocess_image(image):
      return image.select(['B4', 'B3', 'B2']).divide(10000) # Blue, Green, Red, NIR

def fetch_sentinel_image():
  # Get Sentinel-2 image collection
  sentinel = ee.ImageCollection("COPERNICUS/S2") \
              .filterBounds(region) \
              .filterDate('2024-01-01', '2024-12-31') \
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))\
              .map(preprocess_image) \
              .median() \
              # .clip(region) # Get just the region
  print("Sentinel-2 image preprocessed successfully!")
  return sentinel

  # Display metadata
  print(sentinel.getInfo())


# **Fetching Landsat-8 Images (UNNECESSARY?)**



In [ ]:

# Get Landsat-8 image collection
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
            .filterBounds(region) \
            .filterDate('2022-01-01', '2024-12-31') \
            .median()

# Select RGB bands
landsat_rgb = landsat.select(['SR_B4', 'SR_B3', 'SR_B2'])

# Print metadata
print(landsat.getInfo())


{'type': 'Image', 'bands': [{'id': 'SR_B1', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SR_B2', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SR_B3', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SR_B4', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SR_B5', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SR_B6', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SR_B7', 'data_type': {

# **Vizulaize the Extraacted images**

In [ ]:

Map = geemap.Map(center = [5.2, -2.1], zoom=10)

Map.addLayer(sentinel, {'min': 0, 'max': 3000}, "Sentinel-2 RGB")
Map.addLayer(landsat_rgb, {'min': 0, 'max': 3000}, "Landsat-8")

#Lets see it
Map

Map(center=[5.2, -2.1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(c…

# **Export images to Google Drive**

In [ ]:
def export_image_to_drive(image, description='Sentinel-2_Mining_Area', folder = 'GEE_Exports'):
  export_task = ee.batch.Export.image.toDrive(
      image=image,
      description=description,
      scale=30,
      region=region,
      fileFormat='GeoTIFF',
      folder= folder  # This is where the file will be saved in Drive
  )
  # Start the export task
  export_task.start()
  print("Export started! Check Google Drive → GEE_Exports folder in 5-10 minutes.")



task_list = ee.batch.Task.list()
for task in task_list:
    print(task.status())




{'state': 'FAILED', 'description': 'Sentinel2_Mining_Area', 'priority': 100, 'creation_timestamp_ms': 1743971429040, 'update_timestamp_ms': 1743971437018, 'start_timestamp_ms': 1743971432868, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': 'Export too large: specified 424957650 pixels (max: 100000000). Specify higher maxPixels value if you intend to export a large area.', 'id': 'LJCRBS2CXAL5OBUJLSNPC3D7', 'name': 'projects/ee-agamaamanda/operations/LJCRBS2CXAL5OBUJLSNPC3D7'}
{'state': 'COMPLETED', 'description': 'Sentinel2_Mining_Area', 'priority': 100, 'creation_timestamp_ms': 1743967216738, 'update_timestamp_ms': 1743967284244, 'start_timestamp_ms': 1743967220943, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1wWAF0vOQZrIwjOIpTQUW3lvEB9Zs7EIo'], 'attempt': 1, 'batch_eecu_usage_seconds': 85.83586120605469, 'id': 'F35E3D6H4OHK3LFVHKUPYB3P', 'name': 'projects/ee-agamaamanda/operations/F35E3D6H4OHK3LFVHKUPYB3P'}
{'state': 'COMPLETED', 'd

# **Moves a file from Google Drive to Colab**

In [ ]:
def move_file_from_drive_to_colab(source_path_on_drive, destination_path_on_colab):
    """Moves a file from Google Drive to Colab."""
    if IN_COLAB:
        source_path = f"/content/drive/My Drive/{source_path_on_drive}"
        try:
            shutil.copy(source_path, destination_path_on_colab)
            print(f"File moved to Colab: {destination_path_on_colab}")
            return True
        except FileNotFoundError:
            print(f"Error: Source file not found at {source_path}")
            return False
        except Exception as e:
            print(f"Error moving file: {e}")
            return False
    else:
        print("Not running in Colab, skipping file move.")
        return False

local_image_path = '/content/Sentinel2_Mining_Area.tif' # Assuming you have downloaded or moved the image


# **Download & Label for training**

In [ ]:
import os

# Check if file exists in Google Colab
print(os.listdir('/content/'))


['.config', 'Sentinel2_Mining_Area.tif', 'patches', 'mining_dataset.csv', 'drive', 'sample_data']


For training purposes, we used images from the ___ satellite:

3-4 meters resolution per pixel
four channels: RGB and near infrared (NIR)
In total, we identified and labeled ASM sites in ~100 satellite images. Because of the difficulty of the task the labels had to be provided by an expert from RWTH Aachen. Since the original images were huge, we chopped them into smaller patches before feeding them into the segmentation model, so that the training data set consisted of more than 15,000 labelled images, 256-by-256 pixels each.

To make both the annotator's and the model's job easier, we computed two more channels from the four already existing ones

In [ ]:
try:
  with rasterio.open(local_image_path) as src:
    image_array = src.read()
    image_metadata = src.profile
    red_band = image_array[0].astype(np.float32)
    green_band = image_array[1].astype(np.float32)
    blue_band = image_array[2].astype(np.float32)
    # nir_band = image_array[3]
except Exception as e:
    print(f"Error opening the image: {e}")

print(image_array.shape)
  # Print metadta
print(image_metadata)


(3, 1114, 1114)
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 1114, 'height': 1114, 'count': 3, 'crs': CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'), 'transform': Affine(0.00026949458523585647, 0.0, -2.1001713027430293,
       0.0, -0.00026949458523585647, 5.500114990078595), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}


# **U-Net Model Definition**

In [ ]:
from logging import log
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.inc = DoubleConv(in_channels, 64)
        self.outc = nn.Conv2d(64, out_channels, kernel_size=1) # Simplified output

    def forward(self, x):
        x1 = self.inc(x)
        logits = self.outc(x1)
        return logits

# Load a pre-trained model (replace with the actual path to your trained model)
n_channels = 3
n_classes = 1
model = UNet(n_channels, n_classes)
try:
    model.load_state_dict(torch.load('/content/drive/My Drive/galamsey_unet_simplified.pth', map_location=torch.device('cpu'))) # Load on CPU for simplicity
    model.eval()
    print("Pre-trained U-Net model loaded successfully!")
except FileNotFoundError:
    print("Error: Pre-trained U-Net model not found. Please train and save your model.")
    model = None

# Define image transformation for prediction
predict_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


Error: Pre-trained U-Net model not found. Please train and save your model.


# **Prediction Function**

In [ ]:
def predict_mining_site(image_array, model, transform, device='cpu'):
    """
    Performs prediction on an image using the U-Net model.

    Args:
        image_array (numpy.ndarray): The input image array (e.g., RGB).
        model (torch.nn.Module): The trained U-Net model.
        transform (torch.transforms): The image transformation pipeline.
        device (str): 'cpu' or 'cuda'.

    Returns:
        numpy.ndarray or None: The binary prediction mask (1 for mining, 0 otherwise), or None if model is not loaded.
    """
    if model is None or image_array is None:
        print("Error: Model not loaded or image not available for prediction.")
        return None

    # Assuming image_array is (bands, height, width), we need (height, width, bands) for PIL
    rgb_image = np.transpose(image_array[:3, :, :], (1, 2, 0))
    pil_image = Image.fromarray(rgb_image).convert('RGB')
    input_tensor = transform(pil_image).unsqueeze(0).to(device) # Add batch dimension

    with torch.no_grad():
        output = model(input_tensor)
        prediction = torch.sigmoid(output).squeeze().cpu().numpy() > 0.5

    return prediction.astype(np.uint8)

# **Connected Components Analysis**

In [ ]:
def analyze_mining_clusters(prediction_mask):
    """
    Analyzes a binary prediction mask to detect clusters of illegal mining activity.

    Args:
        prediction_mask (numpy.ndarray): A binary mask where 1 represents mining.

    Returns:
        list: A list of dictionaries, where each dictionary contains information
              about a detected mining cluster (e.g., area, bounding box, centroid).
    """
    if prediction_mask is None:
        print("Error: No prediction mask available for cluster analysis.")
        return []

    labeled_mask = label(prediction_mask, connectivity=1)
    regions = regionprops(labeled_mask)
    mining_clusters = []
    for region in regions:
        if region.area > 10: # Example threshold
            mining_clusters.append({
                'area': region.area,
                'bounding_box': region.bbox,
                'centroid': region.centroid
            })
    return mining_clusters


# **Basic Risk Propagation (Conceptual)**

In [ ]:
def calculate_risk_propagation(elevation_data, mining_locations, water_sources):
    """
    A very basic conceptual implementation of risk propagation using Dijkstra's.
    This requires actual elevation data and water source locations.

    Args:
        elevation_data (numpy.ndarray): A 2D array representing elevation.
        mining_locations (list of tuples): Coordinates (row, col) of mining sites.
        water_sources (list of tuples): Coordinates (row, col) of water sources.

    Returns:
        dict: A dictionary mapping mining locations to the nearest water source path (conceptual).
    """
    if elevation_data is None or not mining_locations or not water_sources:
        print("Error: Missing data for risk propagation analysis.")
        return {}

    rows, cols = elevation_data.shape
    graph = nx.grid_2d_graph(rows, cols)

    # Define a very simple weight based on elevation (higher elevation = higher weight)
    for u, v, data in graph.edges(data=True):
        u_row, u_col = u
        v_row, v_col = v
        weight = (elevation_data[u_row, u_col] + elevation_data[v_row, v_col]) / 2 + 1
        data['weight'] = weight

    risk_paths = {}
    for mining_loc in mining_locations:
        shortest_path = None
        min_distance = float('inf')
        for water_loc in water_sources:
            try:
                path = nx.dijkstra_path(graph, source=mining_loc, target=water_loc, weight='weight')
                distance = nx.dijkstra_path_length(graph, source=mining_loc, target=water_loc, weight='weight')
                if distance < min_distance:
                    min_distance = distance
                    shortest_path = path
            except nx.NetworkXNoPath:
                pass

        if shortest_path:
            risk_paths[mining_loc] = f"Path to nearest water source for {mining_loc}" # Conceptual path

    return risk_paths

# Create dummy elevation data and water sources for demonstration
if image_array is not None:
    dummy_elevation_data = np.random.rand(image_array.shape[1], image_array.shape[2]) * 100
    dummy_water_sources = [(50, 50), (150, 200)] # Example coordinates

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 10)

# **Flask Web Application**

In [ ]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/analyze', methods=['POST'])
def analyze_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image part in the request'}), 400
    file = request.files['image']
    if file.filename == '':
        return jsonify({'error': 'No selected image'}), 400

    try:
        # Save the uploaded image temporarily
        uploaded_image_path = 'uploaded_image.tif'
        file.save(uploaded_image_path)
        with rasterio.open(uploaded_image_path) as src:
            uploaded_image_array = src.read()

        prediction_mask = predict_mining_site(uploaded_image_array, model, predict_transform)
        if prediction_mask is not None:
            mining_clusters = analyze_mining_clusters(prediction_mask)
            mining_locations = [cluster['centroid'] for cluster in mining_clusters]

            risk_paths = {}
            if 'dummy_elevation_data' in globals() and 'dummy_water_sources' in globals():
                risk_paths = calculate_risk_propagation(dummy_elevation_data, mining_locations, dummy_water_sources)

            return jsonify({
                'status': 'success',
                'mining_clusters': mining_clusters,
                'risk_paths': risk_paths
            })
        else:
            return jsonify({'error': 'Error during mining site prediction'}), 500

    except Exception as e:
        return jsonify({'error': str(e)}), 500
    finally:
        if os.path.exists('uploaded_image.tif'):
            os.remove('uploaded_image.tif')

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


# **Visualize Sattelitte Image**

In [ ]:
import matplotlib.pyplot as plt

# def normalize(array):
#     return ((array - array.min()) / (array.max() - array.min()))
def normalize(band):
    band = band.astype(np.float32)
    return 255 * (band - band.min()) / (band.max() - band.min())


#lets extract the RGB
red_band = normalize(image_array[0])
green_band = normalize(image_array[1])
blue_band = normalize(image_array[2])

# Create an RGB image
rgb_img = np.dstack((red_band, green_band, blue_band))
PATCH_SIZE = 256
H, W, _ = rgb_img.shape
print(f"Loaded image with shape: {H} x {W}")

# Display the RGB image
plt.figure(figsize=(10, 10))
plt.imshow(rgb_img)
plt.title('Sentinel- 2 Mining RGB img')
plt.axis('off')
plt.show()

# Create patch output folder
patch_folder = "patches/images"
os.makedirs(patch_folder, exist_ok=True)

# Slice image into patches
patch_id = 0
for i in range(0, H, PATCH_SIZE):
    for j in range(0, W, PATCH_SIZE):
        patch = rgb_img[i:i+PATCH_SIZE, j:j+PATCH_SIZE]

        #only save full patches
        if patch.shape[0] != PATCH_SIZE or patch.shape[1] != PATCH_SIZE:
          continue

        #save patch
        patch_img = Image.fromarray(patch.astype(np.uint8))
        patch_img.save(os.path.join(patch_folder, f"patch_{patch_id}.png"))
        patch_id += 1
print(f"Saved patch {patch_id} with shape: {patch.shape}")



# **Pre-process Image**

1. Convert image to grayscale
2. Remove clouds and shadows ->  NDVI-based filtering
3. Enhance image contrast (Histogram Equalization)
4. Extract features

In [ ]:
from skimage.color import rgb2gray

gray_scale = rgb2gray(rgb_img)

plt.figure(figsize=(10, 10))
plt.imshow(gray_scale, cmap='gray')
plt.title('Sentinel- 2 Mining Grayscale img')
plt.axis("off")
plt.show()

# **Remove the clouds & make image clear**
NOTE: mining activities in the study area usually cause a **removal of vegetation and thecreation of dug-outs**.

Normalized Difference Vegetation Index (**NDVI**) helps detect vegetation vs. non-vegetation areas:

$$NDIV = \frac{NIR - RED}{NIR +RED}$$



*   *NIR (Near-Infrared Band, B8)* = High for vegetation
*   *RED (B4)* = Low for vegetation
* Clouds & shadows usually have*NDVI near 0 or negative*



In [ ]:
nir_band = image_array[2].astype(np.float32)
red_band = image_array[0].astype(np.float32)

# Calculate NDVI
ndvi = (nir_band - red_band) / (nir_band + red_band)

#show NDVI image
plt.figure(figsize=(10, 10))
plt.imshow(ndvi, cmap='RdYlGn') # Green = vegetation, Red = bare land/ mining
plt.title('Sentinel- 2 Mining NDVI img')
plt.axis("off")
plt.show()

In [ ]:
from skimage import exposure

# Apply histogram equalization
equalized_image = exposure.equalize_hist(gray_scale)

# Display
plt.figure(figsize=(8, 8))
plt.imshow(equalized_image, cmap="gray")
plt.title("Contrast Enhanced Image")
plt.axis("off")
plt.show()


# **Extract Features**

In [ ]:
# Compute NDWI
ndwi = (green_band - nir_band) / (green_band + nir_band)

# Display NDWI
plt.figure(figsize=(8, 8))
plt.imshow(ndwi, cmap="Blues")  # Blue = Water, White = Mining
plt.title("NDWI - Water Detection")
plt.colorbar()
plt.axis("off")
plt.show()


In [ ]:
from skimage.feature import graycomatrix, graycoprops

# Compute GLCM (Gray Level Co-occurrence Matrix)
glcm = graycomatrix((gray_scale * 255).astype(np.uint8), distances=[5], angles=[0], levels=256, symmetric=True, normed=True)

# Extract contrast and homogeneity
contrast = graycoprops(glcm, 'contrast')[0, 0]
homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]

print(f"GLCM Contrast: {contrast}")
print(f"GLCM Homogeneity: {homogeneity}")


# **Prepare Data for ML**

 Store in tabular format:  1 row = 1 pixel of the image


---



 Red Band (B4)	  - land cover differences

 Green Band (B3) - vegetation

 Blue Band (B2) -	Detects water bodies

 NDVI	 -  vegetation health inidicator

 NDWI	-  water pollution indicator(detect mining impacts)

 GLCM Contrast - land texture changes (rougher textures @ mining sites)

 GLCM Homogeneity	 -  uniformity of land surfaces


In [ ]:
import pandas as pd


# flattedn image bands to 1D arrays
red_val = image_array[0].flatten()
green_val = image_array[1].flatten()
blue_val = image_array[2].flatten()
ndvi_val = ndvi.flatten()
ndwi_val = ndwi.flatten()

#create df
df = pd.DataFrame({
    "Red": red_val,
    "Green": green_val,
    "Blue": blue_val,
    "NDVI": ndvi_val,
    "NDWI": ndwi_val,
    "GLCM_Contrast": contrast,
    "GLCM_Homogeneity": homogeneity,
    "Mining_Site": None
})
print(df.head())

# import ace_tools as tools
# tools.display_dataframe_to_user(name="Extracted Features Dataset", dataframe=df)

# **Try: Supervised learing**
label half as mining areas & half as non mining

In [ ]:
df.loc[:250, "Mining_Site"] = 1
df.loc[251:500, "Mining_Site"] = 0

# Drop  None
df = df.dropna()

# Display updated dataset
print(df.head())

# Save dataset to CSV
csv_path = "/content/mining_dataset.csv"
df.to_csv(csv_path, index=False)

print(f"Dataset saved at: {csv_path}")


# Check if file exists in Google Colab
print(os.listdir('/content/'))



# **Training ML Model**

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=["Mining_Site"])
y = df["Mining_Site"] #taget var

#80/20  train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")